# 使用 LCEL 构建一个简单的 LLM 应用

In [30]:
import getpass
import os
import langchain as lc

# 禁用 LangSmith 追踪（如果不需要）
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "llm-langchain-learn"
# 设置 OpenAI API 密钥（阿里云）
os.environ["OPENAI_API_KEY"] = getpass.getpass("请输入阿里云 API Key: ")

In [31]:
from langchain_openai import ChatOpenAI

# 选择模型
model = ChatOpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    model="qwen-plus"
)

In [32]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that translates Chinese to English."),
    HumanMessage(content="Translate this sentence from Chinese to English: '我爱编程!'"),
]
model.invoke(messages)

AIMessage(content='I love programming!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 37, 'total_tokens': 41, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-ae359788-ef1a-9dfe-86d5-25f74568b6ac', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c9c452ab-f602-4806-b8b9-35b63454d473-0', usage_metadata={'input_tokens': 37, 'output_tokens': 4, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

## 输出解析器

In [24]:
# 使用解析器解析输出
from langchain_core.output_parsers import StrOutputParser
# 构造解析器
parser = StrOutputParser()
# 使用1:单独使用
result = model.invoke(messages)
parser.invoke(result)

# 使用2:与链结合使用
chain = model | parser
chain.invoke(messages)  # 直接返回解析后的字符串

'I love programming!'

## 提示词模版-使用 LCEL 连接组件

In [35]:
from langchain.prompts import ChatPromptTemplate
# 构造提示模板
system_template = "You are a helpful assistant that translates English to {target_language}."

# PromptTemplate,这将是 system_template 和一个更简单的模板的组合，用于放置要翻译的文本
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "Translate this sentence from Chinese to {target_language}: '{text_to_translate}'"),
])
# 使用1:提示模板构造消息
messages = prompt_template.format_messages(
    target_language="French",
    text_to_translate="我爱编程!"
)
model.invoke(messages)  # 调用模型

# 使用2:链结合提示模板和模型
chain = prompt_template | model | parser
chain.invoke(input={
    "target_language": "French",
    "text_to_translate": "我爱编程!"
})

"J'adore la programmation !"

## 使用 LangServe 提供服务## 

In [ ]:
# !pip install "langserve[all]"



   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 60.7 MB/s eta 0:00:00
